# Naive Bayes

El ingenuo de bayes es una regla basada en la regla de Bayes para determinar la probabilidad de un evento, dado que ya ocurrieron otros eventos.

La forma simple de la probabilidad para un evento `A` se termina contando cuántas veces aparece ese evento, entre el total de eventos posibles.

> Ecuación 1.1

$P(A) - Significa cuántas veces ocurre A en el espacio muestral$

Por ejemplo, el evento `A` puede ser:

* A - CFP será mayor al promedio
* A - ESFUERZO_TOTAL_P > ESFUERZO_TOTAL_R
* A - La palabra "líder" aparece en las observaciones
* A - Si el proyecto tardó más de 30 días en concluir

Entonces los eventos representan generalmente afirmaciones sobre nuestros datos, es decir, eventos que pueden ocurrir en nuestros datos, como que un proyecto tenga ciertas características. Incluso el evento podría ser más complejo. Por ejemplo, `A - COVAR(CFP, PDR) > 0.5`.

Entonces, los eventos hablan de verdades sobre nuestros datos, y estas características nos permiten lanzar conclusiones sobre cómo están nuestros datos.

Por ejemplo, sabiendo que `A - EFICIENCIA será mayor a 0.8`, podríamos determinar la probabilidad que este evento ocurra, es decir, cuántas veces ocurre el evento `A`.

> Ecuación 1.2

$P(A) = (total sí A) / total$

Las probabilidades no hablan de la frecuencia o las veces por razón que puede ocurrir el evento, por ejemplo, `P(A) = 1004 / 2995 ~= 0.334`, significaría que el evento `A` ocurre con una frecuencia del `33.4%`, entonces, podemos nosotros pensar que si llegan nuevos datos, habrá un `33.4%` que el proyecto cumpla el evento (que la eficiencia sea mayor a 0.8).

Cuando tenemos muchos eventos, queremos determinar probabilidades ocultas, para poder entender mejor nuestros datos, por ejemplo, en dos eventos.

- A - CFP será mayor al promedio
- B - EFICIENCIA será mayor a 0.8

La pregunta probabilística para dos eventos es:

    P(B | A) - Cuál es la probabilidad que ocurra B
                dado que ya ocurrió el evento A

Es decir, preguntamos: ¿Qué probabilidad hay que la eficiencia sea mayor a 0.8, sabiendo que el CFP es mayor al promedio?

La regla de bayes dice:

> Ecuación 1.3

$P(B | A) = P(A | B) P(B) / P(A)$

> Ecuación 1.4

$P(A | B) = P(B | A) P(A) / P(B)$

La respuesta a `P(A | B)` o `P(B | A)` debería conocerse a priori, es decir, con un histórico o usando conteos deberíamos tener alguna de estas dos probabilidades, para determinar la otra.

> Ecuación 1.5

$P(A) - La conozco por un conteo simple$

> Ecuación 1.6

$P(B) - La conozco por un conteo simple$

> Ecuación 1.7

$P(A | B) - La conozco por un conteo sobre un histórico$

A partir de aquí podemos inferir la otra probabilidad.

La regla generalizada de bayes se conoce como el Naive Bayes.

> Ecuación 2.1

$P(y | x1, x2, x3, ...) = p(x1, x2, x3, ... | y) P(x1, x2, x3, ...) / P(y)$

> Ecuación 2.2

$P(y | x1, x2, x3, ...) ~= p(x1 | y) p(x2 | y) p(x3 | y) ... $

## Ejemplo

In [12]:
import pandas as pd

dataset = pd.DataFrame({
    "A": [1, 2, 2, 1, 1, 3, 2, 1, 3, 3, 4, 5, 2],
    "B": [0.6, 0.7, 0.4, 0.5, 0.75, 0.76, 0.8, 0.5, 0.6, 1.0, 0.96, 0.93, 0.64]
})

dataset

,A,B
0,1,0.60
1,2,0.70
2,2,0.40
3,1,0.50
4,1,0.75
5,3,0.76
6,2,0.80
7,1,0.50
8,3,0.60
9,3,1.00


In [13]:
dataset["EVENTO_A"] = dataset["A"] >= 3
dataset["EVENTO_B"] = dataset["B"] >= 0.7

dataset

,A,B,EVENTO_A,EVENTO_B
0,1,0.60,False,False
1,2,0.70,False,True
2,2,0.40,False,False
3,1,0.50,False,False
4,1,0.75,False,True
5,3,0.76,True,True
6,2,0.80,False,True
7,1,0.50,False,False
8,3,0.60,True,False
9,3,1.00,True,True


In [14]:
pA = (dataset[ (dataset["EVENTO_A"]) ].count() / dataset.count()).values[0]

pB = (dataset[ (dataset["EVENTO_B"]) ].count() / dataset.count()).values[0]

pA, pB

(0.38461538461538464, 0.5384615384615384)

In [15]:
# Alternativamente: Cómo los eventos son de Bernoulli (1 o 0)

dataset["EVENTO_A"].mean(), dataset["EVENTO_B"].mean()

(0.38461538461538464, 0.5384615384615384)

In [16]:
# Por calcular P(B | A) = P(A | B) P(B) / P(A)

# Calcula: P(A | B) mediante el análisis de afinidad

total = dataset[ (dataset["EVENTO_B"]) ].count().values[0]
support = dataset[ (dataset["EVENTO_A"]) & (dataset["EVENTO_B"]) ].count().values[0]

p_A_B = support / total

print("La probabilidad que la variable A >= 3 dado que la variable B >= 0.7")

p_A_B * 100

La probabilidad que la variable A >= 3 dado que la variable B >= 0.7


57.14285714285714

Esta probabilidad determina con qué frecuencia ocurre el evento A, dado que el evento B ya ocurrió. Pero nosotros queremos la inversa, porque la variable A es a priori, significa que en el futuro o la actualidad, nosotros desconocemos el valor de la variable B (eficiencia), pero si conocemos el valor de A (CFP)

In [17]:
# Calcular: P(B | A) = P(A | B) P(B) / P(A)

# A - PRIORI
# B - POSTERIORI

p_B_A = p_A_B * pB / pA

p_B_A

0.7999999999999998

## Análisis real sobre los proyectos

In [19]:
import pandas as pd

proyectos = pd.read_excel("data/ProyectosFinalizados.xlsx", sheet_name="base")

def limpiar_columna(columna):
    # 1. Reemplazar caracteres especiales
    columna = columna.replace(" ", "_") \
        .replace("á", "a") \
        .replace("é", "e") \
        .replace("í", "i") \
        .replace("ó", "o") \
        .replace("ú", "u") \
        .replace("ñ", "n") \
        .replace("+", "_") \
        .replace(".", "_") \
        .replace("(", "_") \
        .replace(")", "")
    import re
    # 2. Poner un guion bajo entre cambio de letras de minúscula a mayúsculas
    columna = re.sub("([a-z])([A-Z])", "\\1_\\2", columna)
    # 3. Poner un guion bajo entre ID y letra
    columna = re.sub("ID([A-Za-z])", "ID_\\1", columna)
    # 4. Poner un guion bajo entre número y letra
    columna = re.sub("([0-9])([A-Za-z])", "\\1_\\2", columna)
    # 4. Poner un guion bajo entre letra y número
    columna = re.sub("([A-Za-z])([0-9])", "\\1_\\2", columna)
    # 5. Quitar un guion bajo entre letra y número
    columna = re.sub("([A-Za-z])_([0-9])", "\\1\\2", columna)
    # 6. Poner un guion bajo entre letra y número final
    columna = re.sub("([A-Za-z])([0-9])$", "\\1_\\2", columna)
    # 7. Convertir a mayúsculas
    columna = columna.upper()
    return columna

proyectos.columns = map(limpiar_columna, proyectos.columns.values)

proyectos.head()

,ID_PROYECTO,EMPRESA,PETICION,PETICION_1,PET_EMPRESA,NOMBRE,CFP,EFICIENCIA,PDR,ESFUERZO_TOTAL_P,...,PDR_S,PDR_M_1,PDR_M_2,PDR_L,LEAD_TIME,F1_GESTIONDELA_DEMANDA,F2_ELICITACION,F3_CONTRUCCION,F4_FINAL,TAMANO
0,1,México,8318.0,8318,8318México,Modificación a la utileria de generación de pa...,12,NaN,33.708333,404.5,...,NaN,NaN,NaN,NaN,914.0,825.0,8.0,42.0,35.0,10-29
1,2,México,9619.0,9619,9619México,Modificacion al programa Vbncarga.exe,2,NaN,147.500000,295.0,...,NaN,NaN,NaN,NaN,609.0,584.0,7.0,14.0,1.0,2-9
2,3,México,10438.0,10438,10438México,Permisos para Cambios de puntos de Sembrado- P...,11,NaN,51.375455,1019.0,...,NaN,NaN,NaN,NaN,519.0,461.0,2.0,37.0,1.0,10-29
3,4,México,10522.0,10522,10522México,Corrección del informe TabuladoCredito,11,NaN,47.727273,525.0,...,NaN,NaN,NaN,NaN,493.0,455.0,9.0,27.0,1.0,10-29
4,5,México,10528.0,10528,10528México,Corrección al Tabulado de Carteras y Hojas men...,4,NaN,26.290000,213.0,...,NaN,NaN,NaN,NaN,557.0,520.0,5.0,30.0,0.0,2-9


## Diseño de eventos

    A - El CFP es mayor al quantil 75%
    B - La eficiencia es menor al promedio

In [21]:
dataset = proyectos[[ "CFP", "EFICIENCIA" ]].dropna()

dataset.head()

,CFP,EFICIENCIA
886,66,0.836066
888,21,0.245455
889,18,0.650492
890,212,0.801831
893,6,0.349970


In [22]:
import numpy as np

# import library
# ...
# library.foo()
# library.bar()

# import library as lib
# ...
# lib.foo()
# lib.bar()

# import library.sublibrary
# ...
# library.sublibrary.zoo()
# library.sublibrary.zip()

# import library.sublibrary as sub
# ...
# sub.zoo()
# sub.zip()

# from library.sublibrary import zoo, zip
# ...
# zoo()
# zip()

# from sklearn.features_extraction.text import CountVectorizer
# ...
# CountVectorizer()
# CountVectorizer()

dataset["EVENTO_A"] = dataset["CFP"] >= np.quantile(dataset["CFP"], 0.75)
dataset["EVENTO_B"] = dataset["EFICIENCIA"] < dataset["EFICIENCIA"].mean()

dataset.head()

,CFP,EFICIENCIA,EVENTO_A,EVENTO_B
886,66,0.836066,True,False
888,21,0.245455,False,True
889,18,0.650492,False,True
890,212,0.801831,True,True
893,6,0.349970,False,True


In [23]:
print("El CFP tiene que ser mayor a 52.0 y la eficiencia menor a 0.8178...")
np.quantile(dataset["CFP"], 0.75), dataset["EFICIENCIA"].mean()

(52.0, 0.8178825563593339)

In [31]:
(dataset["EVENTO_A"] & dataset["EVENTO_B"]).head()

886    False
888    False
889    False
890     True
893    False
dtype: bool

In [29]:
(dataset["EVENTO_A"] & dataset["EVENTO_B"]).sum() / dataset.count().values[0]

0.11889035667107001

In [32]:
(dataset["EVENTO_A"] & dataset["EVENTO_B"]).mean()

0.11889035667107001

In [37]:
pA = dataset["EVENTO_A"].mean()
pB = dataset["EVENTO_B"].mean()
pA_B = (dataset["EVENTO_A"] & dataset["EVENTO_B"]).mean()

print("La probabilidad que CFP >= 52 es: {:.1f}%".format(pA * 100))
print("La probabilidad que EFICIENCIA < 0.8178... es: {:.1f}%".format(pB * 100))
print("La probabilidad que CFP >= 52 dado que EFICIENCIA < 0.8178 es: {:.1f}%".format(pA_B * 100))

La probabilidad que CFP >= 52 es: 25.4%
La probabilidad que EFICIENCIA < 0.8178... es: 54.7%
La probabilidad que CFP >= 52 dado que EFICIENCIA < 0.8178 es: 11.9%


In [39]:
pA, pB, pA_B

(0.25429326287978865, 0.5468956406869221, 0.11889035667107001)

In [44]:
pA_B * pB / pA

0.2556914683731064

In [41]:
pB_A = pA_B * pB / pA

print("La probabilidad que EFICIENCIA < 0.8178 dado que CFP >= 52 es: {:.1f}%".format(pB_A * 100))

La probabilidad que EFICIENCIA < 0.8178 dado que CFP >= 52 es: 25.6%


In [51]:
dataset["EVENTO_A"] = dataset["CFP"] >= 100
dataset["EVENTO_B"] = dataset["EFICIENCIA"] < dataset["EFICIENCIA"].mean()

pA = dataset["EVENTO_A"].mean()
pB = dataset["EVENTO_B"].mean()
pA_B = (dataset["EVENTO_A"] & dataset["EVENTO_B"]).mean()

print("La probabilidad que CFP >= 100 es: {:.1f}%".format(pA * 100))
print("La probabilidad que EFICIENCIA < 0.8178... es: {:.1f}%".format(pB * 100))
print("La probabilidad que CFP >= 100 dado que EFICIENCIA < 0.8178 es: {:.1f}%".format(pA_B * 100))

pB_A = pA_B * pB / pA

print("La probabilidad que EFICIENCIA < 0.8178 dado que CFP >= 100 es: {:.1f}%".format(pB_A * 100))

La probabilidad que CFP >= 100 es: 12.4%
La probabilidad que EFICIENCIA < 0.8178... es: 54.7%
La probabilidad que CFP >= 100 dado que EFICIENCIA < 0.8178 es: 5.2%
La probabilidad que EFICIENCIA < 0.8178 dado que CFP >= 100 es: 22.7%


In [66]:
dataset = proyectos[[ "ESFUERZO_TOTAL_R", "ESFUERZO_TOTAL_P", "EFICIENCIA" ]].dropna()

dataset["EVENTO_A"] = dataset["ESFUERZO_TOTAL_R"] - dataset["ESFUERZO_TOTAL_P"] > 1000
dataset["EVENTO_B"] = dataset["EFICIENCIA"] < dataset["EFICIENCIA"].mean()

pA = dataset["EVENTO_A"].mean()
pB = dataset["EVENTO_B"].mean()
pA_B = (dataset["EVENTO_A"] & dataset["EVENTO_B"]).mean()

print("La probabilidad que ESFUERZO TOTAL REAL difiera más de 10 días respecto al Planificado es: {:.1f}%".format(pA * 100))
print("La probabilidad que EFICIENCIA < 0.8... es: {:.1f}%".format(pB * 100))
print("La probabilidad que ESFUERZO TOTAL REAL difiera más de 10 días respecto al Planificado dado que EFICIENCIA < 0.8 es: {:.1f}%".format(pA_B * 100))

pB_A = pA_B * pB / pA

print("La probabilidad que EFICIENCIA < 0.8 dado que ESFUERZO TOTAL REAL difiera más de 10 días respecto al Planificado es: {:.1f}%".format(pB_A * 100))

La probabilidad que ESFUERZO TOTAL REAL difiera más de 10 días respecto al Planificado es: 3.8%
La probabilidad que EFICIENCIA < 0.8... es: 54.7%
La probabilidad que ESFUERZO TOTAL REAL difiera más de 10 días respecto al Planificado dado que EFICIENCIA < 0.8 es: 3.2%
La probabilidad que EFICIENCIA < 0.8 dado que ESFUERZO TOTAL REAL difiera más de 10 días respecto al Planificado es: 45.3%


In [54]:
np.quantile(dataset["EFICIENCIA"], 0.7)

1.0915658284045127